## Pipeline final per ajuntar tots els models
- Suposem un input i fem tot el proccès fin a tenir l'output final.
- Serà el fluxe que es seguirà per tractar la petició del client al servidor.

#### Llibreries necessàries:

In [1]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
from PIL import Image
import pytesseract

#### Rutes dels models:

In [2]:
yolo_model = './bbox_detection/runs/detect/train3_10k_22e/weights/last.pt'
image_bytes = '../generador-tiquets/script_general/exports/data/complexity_5/coco/tickets/000003.png'

#### Carreguem els models:

In [3]:
model = YOLO(yolo_model)

#### Detecció bbox:

In [4]:
inference = model(image_bytes)


image 1/1 /home/jaume/tiquets/analisis-tiquets/ocr-tiquets/../generador-tiquets/script_general/exports/data/complexity_5/coco/tickets/000003.png: 640x352 8 descripcios, 6 quantitat_uds, 2 quantitat_kgs, 8 imports, 1 total, 5 preu_unitaris, 28.7ms
Speed: 1.2ms preprocess, 28.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 352)


In [5]:
data = []

data_boxes = inference[0].boxes.data

data_boxes_np = data_boxes.cpu().detach().numpy()

for bbx in data_boxes_np.tolist():
    data.append({
      'x1': bbx[0],
      'y1': bbx[1],
      'x2': bbx[2],
      'y2': bbx[3],
      'acc': bbx[4],
      'label': bbx[5]
    })

In [6]:
predicts = pd.DataFrame(data)
predicts

,x1,y1,x2,y2,acc,label
0,64.419746,410.086761,264.449005,430.064148,0.983361,0.0
1,64.842064,350.071442,264.379395,369.972473,0.981628,0.0
2,64.592522,470.086517,264.815491,490.080444,0.978943,0.0
3,64.410973,450.137756,264.764282,470.145325,0.975177,0.0
4,64.826500,390.102356,264.467377,410.114105,0.974609,0.0
5,64.752174,369.979462,264.790588,390.023041,0.974204,0.0
6,25.032227,470.057220,64.395721,489.980804,0.972065,1.0
7,25.062021,449.979736,64.468971,469.984985,0.971747,1.0
8,25.091209,390.078705,64.958710,410.071198,0.971529,1.0
9,25.046497,350.114838,64.740570,369.964264,0.970886,1.0


In [7]:
predicts['label_name'] = predicts['label'].map(inference[0].names)
predicts.head(2)

,x1,y1,x2,y2,acc,label,label_name
0,64.419746,410.086761,264.449005,430.064148,0.983361,0.0,descripcio
1,64.842064,350.071442,264.379395,369.972473,0.981628,0.0,descripcio


#### Treiem el text que hi ha dins de cada bbox:
- Crear el bbox a partir de les coordenades.
- Fem un crop de la imatge amb la llibreria pillow.
- Fem inferència sobre aquest retall per saber que posa a cada bbox predit.

In [8]:
predicts['b_1'] = predicts.apply(lambda row: (row.x1, row.y1), axis = 1)
predicts['b_4'] = predicts.apply(lambda row: (row.x2, row.y2), axis = 1)
predicts.head(2)

,x1,y1,x2,y2,acc,label,label_name,b_1,b_4
0,64.419746,410.086761,264.449005,430.064148,0.983361,0.0,descripcio,"(64.41974639892578, 410.0867614746094)","(264.4490051269531, 430.06414794921875)"
1,64.842064,350.071442,264.379395,369.972473,0.981628,0.0,descripcio,"(64.8420639038086, 350.0714416503906)","(264.37939453125, 369.97247314453125)"


In [9]:
pilImg = Image.open(image_bytes)

def get_text(row):
    cropped_img = pilImg.crop((row.b_1[0], row.b_1[1], row.b_4[0], row.b_4[1]))

    img = np.array(cropped_img)
    text = pytesseract.image_to_string(img, config="--psm 7")

    return text

predicts['value'] = predicts.apply(get_text, axis = 1)
predicts

,x1,y1,x2,y2,acc,label,label_name,b_1,b_4,value
0,64.419746,410.086761,264.449005,430.064148,0.983361,0.0,descripcio,"(64.41974639892578, 410.0867614746094)","(264.4490051269531, 430.06414794921875)",ESPINAC FULLA
1,64.842064,350.071442,264.379395,369.972473,0.981628,0.0,descripcio,"(64.8420639038086, 350.0714416503906)","(264.37939453125, 369.97247314453125)",LLAMANTOL AMERICA
2,64.592522,470.086517,264.815491,490.080444,0.978943,0.0,descripcio,"(64.59252166748047, 470.0865173339844)","(264.81549072265625, 490.0804443359375)",ESPAGUETI ARMANDO.
3,64.410973,450.137756,264.764282,470.145325,0.975177,0.0,descripcio,"(64.41097259521484, 450.13775634765625)","(264.7642822265625, 470.14532470703125)",MONGETA CUITA
4,64.826500,390.102356,264.467377,410.114105,0.974609,0.0,descripcio,"(64.82649993896484, 390.10235595703125)","(264.4673767089844, 410.1141052246094)",COLORACIO
5,64.752174,369.979462,264.790588,390.023041,0.974204,0.0,descripcio,"(64.7521743774414, 369.9794616699219)","(264.79058837890625, 390.0230407714844)",GAMBOT CONGELAT
6,25.032227,470.057220,64.395721,489.980804,0.972065,1.0,quantitat_ud,"(25.0322265625, 470.0572204589844)","(64.39572143554688, 489.9808044433594)",2
7,25.062021,449.979736,64.468971,469.984985,0.971747,1.0,quantitat_ud,"(25.062021255493164, 449.979736328125)","(64.4689712524414, 469.9849853515625)",1
8,25.091209,390.078705,64.958710,410.071198,0.971529,1.0,quantitat_ud,"(25.091209411621094, 390.0787048339844)","(64.95870971679688, 410.0711975097656)",2
9,25.046497,350.114838,64.740570,369.964264,0.970886,1.0,quantitat_ud,"(25.046497344970703, 350.1148376464844)","(64.74057006835938, 369.9642639160156)",1


#### Relacionem descripció amb quantitat, preu_unitari i total:
- Amb el linear_sum_assignment mirem a quina descripció correspon cada quantitat_ud, quantitat_kg, preu_unitari, import
- Això ens relaciona els index.
- Després hem de relacionar-ho amb el dataframe original per saber el valor.

In [10]:
ticket_info = {'ticket_id': {}}
ticket_info

{'ticket_id': {}}

In [11]:
def create_cost_matrix(Ytest, Ypred):
        cost_matrix = np.zeros((len(Ytest.values), len(Ypred.values)))

        for i, x in enumerate(Ytest.values):
            for j, y in enumerate(Ypred.values):
                cost_matrix[i, j] = abs(x - y)

        return cost_matrix

def get_relations(idxs_true, idxs_pred):
    relacions = {}

    for i in range(len(idxs_true)):
        idx_descripcio = Ytest[Ytest == Ytest.values[idxs_true[i]]].index[0]
        idx_label = Ypred[Ypred == Ypred.values[idxs_pred[i]]].index[0]

        relacions[idx_descripcio] = idx_label

    return relacions

In [12]:
g_df = predicts[predicts['label_name'] != 'total'].copy()

Ytest = g_df[g_df['label_name'] == 'descripcio'].y1

for label in ['import', 'quantitat_ud', 'quantitat_kg']:
    Ypred = g_df[g_df['label_name'] == label].y1

    if label == 'quantitat_kg':
        Ypred -= 20

    cost_matrix = create_cost_matrix(Ytest, Ypred)

    idxs_true, idxs_pred = linear_sum_assignment(cost_matrix)

    for i in range(len(idxs_true)):
        idx_descripcio = Ytest[Ytest == Ytest.values[idxs_true[i]]].index[0] #[0] perquè retornaba un objecte Index

        # Crear el atribut descripcio_id (que és el id d'aquesta descripció)
        if idx_descripcio not in ticket_info['ticket_id']:
            ticket_info['ticket_id'][idx_descripcio] = {}

        idx_label = Ypred[Ypred == Ypred.values[idxs_pred[i]]].index[0]

        ticket_info['ticket_id'][idx_descripcio][label] = idx_label



# preu unitari dels kg
Ytest = g_df[g_df['label_name'] == 'quantitat_kg'].y1
Ypred = g_df[g_df['label_name'] == 'preu_unitari'].y1

cost_matrix = create_cost_matrix(Ytest, Ypred)
idxs_true, idxs_pred = linear_sum_assignment(cost_matrix)

relacions = get_relations(idxs_true, idxs_pred)


# Això està fora i he tallat el bucle i l'he tornat a començar perquè sinó no em deixava
# modificar l'objecte ticket_info, donava error : RuntimeError: dictionary changed size during iteration
copied_items = dict(ticket_info['ticket_id']).copy()

for k_1, v_1 in relacions.items():
    # Busquem a quina descripció (key del diccionari interior de ticket_info['00000X])
    # Té a dins
    for k_2, v_2 in copied_items.items():
        x = v_2.copy()
        for v_3 in x.values(): # Aquí donava el error
            if v_3 == k_1:
                ticket_info['ticket_id'][k_2]['preu_unitari'] = v_1.copy()

# preu unitari dels ud
preus_unitaris = g_df[g_df['label_name'] == 'preu_unitari'].y1
preus_unitaris_ud = preus_unitaris.drop(relacions.values())

Ytest = g_df[g_df['label_name'] == 'descripcio'].y1
Ypred = preus_unitaris_ud

cost_matrix = create_cost_matrix(Ytest, Ypred)
idxs_true, idxs_pred = linear_sum_assignment(cost_matrix)

for i in range(len(idxs_true)):
    idx_descripcio = Ytest[Ytest == Ytest.values[idxs_true[i]]].index[0] #[0] perquè retornaba un objecte Index

    # Crear el atribut descripcio_id (que és el id d'aquesta descripció)
    if idx_descripcio not in ticket_info['ticket_id']:
        ticket_info['ticket_id'][idx_descripcio] = {}

    idx_label = Ypred[Ypred == Ypred.values[idxs_pred[i]]].index[0]

    ticket_info['ticket_id'][idx_descripcio]['preu_unitari'] = idx_label

ticket_info

{'ticket_id': {0: {'import': 14, 'quantitat_kg': 25, 'preu_unitari': 23},
  1: {'import': 24, 'quantitat_ud': 9},
  2: {'import': 20, 'quantitat_ud': 6, 'preu_unitari': 26},
  3: {'import': 16, 'quantitat_ud': 7},
  4: {'import': 28, 'quantitat_ud': 8, 'preu_unitari': 10},
  5: {'import': 22, 'quantitat_ud': 13, 'preu_unitari': 12},
  11: {'import': 17, 'quantitat_kg': 29, 'preu_unitari': 21},
  15: {'import': 19, 'quantitat_ud': 18}}}

In [13]:
relacions

{25: 23, 29: 21}

In [14]:
data = []

tiquet_obj = ticket_info['ticket_id']

for key, value in tiquet_obj.items():
    descripcio_id = key
    quantitat_ud_id = value.get('quantitat_ud', None)
    quantitat_kg_id = value.get('quantitat_kg', None)
    preu_unitari_id = value.get('preu_unitari', None)
    import_id = value.get('import', None)

    data.append({
        'descripcio': predicts.iloc[descripcio_id].value,
        'quantitat': predicts.iloc[quantitat_ud_id].value if quantitat_ud_id is not None else predicts.iloc[quantitat_kg_id].value,
        'preu_unitari': predicts.iloc[preu_unitari_id].value if preu_unitari_id is not None else predicts.iloc[import_id].value,
        'import': predicts.iloc[import_id].value
    })

df_related_products = pd.DataFrame(data)
df_related_products

,descripcio,quantitat,preu_unitari,import
0,ESPINAC FULLA,0.345 ke,"1,24 €/kg",0.43
1,LLAMANTOL AMERICA,1,"14,30","14,30"
2,ESPAGUETI ARMANDO.,2,1.04,2.09
3,MONGETA CUITA,1,0.65,0.65
4,COLORACIO,2,9.62,19.24
5,GAMBOT CONGELAT,4,"6.855,26","421,02"
6,JULIVERT,0.998 kg,15.98 €/kg,15.95
7,REFRESC AMB,1,1.64,1.64


- Afegime el total en aquest dataframe perquè abans l'hem filtrat.

In [15]:
total_row = predicts[predicts['label_name'] == 'total']
total_row

,x1,y1,x2,y2,acc,label,label_name,b_1,b_4,value
27,429.565002,530.109985,474.062225,550.186462,0.961011,4.0,total,"(429.56500244140625, 530.1099853515625)","(474.0622253417969, 550.1864624023438)","475,31"


#### Montem el `json` que retornarem al usuari i cap a la ETL
- La informació dels ivas, les dades del supermercat i totes les altres dades, les posarem de forma random, ja que detectar-les amb la xarxa neuronal, hagués complicat molt i així ja cobrim els coneixements necessaris pel projecte.

In [16]:
tickets_template = {
    'nom': "MERCADONA, S.A. A-12345678",
    'adressa': 'C/ JOSEP MARÍA FOLCH I TORRES, 5',
    'ciutat': "TARRAGONA 43006",
    'telf': "977598991",
    'data': "22-04-2024",
    'op': "12345",
    'factura_simplificada': "1234-123-12345",
    'barcode': 'image',
    'total_amb_IVA': '', # total aquí
    'pagat_targeta_bancaria': '', # total aquí
    'total_sense_IVA': 'AAA,AA',
    'iva_total': '50,33',
    'targeta_bancaria': "**** **** **** **** 1234",
    'NC': '123456789',
    'AUT': '123456',
    'AID': "B0000000012345",
    'ARC': '1234',
    'targeta_1': 'VISA CREDITO/DEB',
    'targeta_2': 'Visa CaixaBank',
    'logo_bottom': 'image',
    'logo_top': 'image',
    'logo_contact_less': 'image',
    'products': [], # Productes aquí
    'ivas': [
        {
            'valor': '21%',
            'base_imposable':
            '44,01', 'quota':
            '9,24',
        },
        {
            'valor': '10%',
            'base_imposable': '3,48',
            'quota': '0,35'
        },
        {
            'valor': '4%',
            'base_imposable': '0,47',
            'quota': '0,02'
        }
    ]
}

#### Posem els nostres valors sobre els de per defecte (productes i total)

In [17]:
tickets_template['products'] = data
tickets_template['total_amb_IVA'] = total_row.value.values[0]
tickets_template['pagat_targeta_bancaria'] = total_row.value.values[0]

In [18]:
tickets_template

{'nom': 'MERCADONA, S.A. A-12345678',
 'adressa': 'C/ JOSEP MARÍA FOLCH I TORRES, 5',
 'ciutat': 'TARRAGONA 43006',
 'telf': '977598991',
 'data': '22-04-2024',
 'op': '12345',
 'factura_simplificada': '1234-123-12345',
 'barcode': 'image',
 'total_amb_IVA': '475,31',
 'pagat_targeta_bancaria': '475,31',
 'total_sense_IVA': 'AAA,AA',
 'iva_total': '50,33',
 'targeta_bancaria': '**** **** **** **** 1234',
 'NC': '123456789',
 'AUT': '123456',
 'AID': 'B0000000012345',
 'ARC': '1234',
 'targeta_1': 'VISA CREDITO/DEB',
 'targeta_2': 'Visa CaixaBank',
 'logo_bottom': 'image',
 'logo_top': 'image',
 'logo_contact_less': 'image',
 'products': [{'descripcio': 'ESPINAC FULLA',
   'quantitat': '0.345 ke',
   'preu_unitari': '1,24 €/kg',
   'import': '0.43'},
  {'descripcio': 'LLAMANTOL AMERICA',
   'quantitat': '1',
   'preu_unitari': '14,30',
   'import': '14,30'},
  {'descripcio': 'ESPAGUETI ARMANDO.',
   'quantitat': '2',
   'preu_unitari': '1.04',
   'import': '2.09'},
  {'descripcio': 'MON

#### Fem un valor real aleatori per per el nom de la ciutat
- Així al power BI es mostraran dades

In [19]:
cat = ['GIRONA', 'BARCELONA', 'LLEIDA', 'TARRAGONA']
establishments = pd.read_csv("../generador-tiquets/dades/establiments/establishments.csv")
establishments = establishments[establishments['provincia'].isin(cat)]
establishments.head(2)

,id,pais,provincia,localidad,direccion,codigo_postal,telefono,horas_aperturas,horas_cierres,proximos_festivos,parking disponible,latitud,longitud,fecha_inauguracion,fecha creacion registro
227,883185314580,ES,BARCELONA,ABRERA,"C/ MARTORELL, 20",08630,937701244,"['09:00', 'Cerrado', '09:00', '09:00', '09:00'...","['21:00', 'Cerrado', '21:00', '21:00', '21:00'...","['14/04/24-C', '21/04/24-C', '28/04/24-C', '01...",S,41.517462,1.899136,29/11/2008,13-04-2024
228,883185315382,ES,BARCELONA,ARENYS DE MAR,"CTRA. N-II KM. 656,3, S/N",08350,938825090,"['09:00', 'Cerrado', '09:00', '09:00', '09:00'...","['21:00', 'Cerrado', '21:00', '21:00', '21:00'...","['14/04/24-C', '21/04/24-C', '28/04/24-C', '01...",S,41.576083,2.544068,29/11/2013,13-04-2024


In [20]:
supermercat = establishments.sample()

In [22]:
supermercat.localidad.values[0], supermercat.codigo_postal.values[0]

("SANT SADURNÍ D'ANOIA", '08770')